# Prétraitement des données audio

# Importation des packages

In [ ]:
import librosa, librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os
import json
from sklearn.model_selection import train_test_split
import torch as torch
from torch.utils.data import Dataset
from torchsummary import summary

# Connection avec Google Drive

Ajoutez un raccourci de ce dossier à votre google drive :

https://drive.google.com/drive/folders/1NGH6ntk3qH8Odo7q8YxDS0iqV-httZUR?usp=sharing

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
file = "drive/MyDrive/Music_genre_classification/genres_original/pop/pop.00008.wav"

Utilisez la fonction `load` de `librosa` pour charger le son.

In [ ]:
signal, sr = None

Utilisez la fonction `waveshow` de `librosa` pour afficher le son.

In [ ]:
None
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.show()

Utilisez la fonction `fft` de `numpy` pour calculer la fast fourier transform.

In [ ]:
fft = None

Utilisez la fonction `abs` de `numpy` pour obtenir la valeur absolu de `fft` et ainsi obtenir la magnitude de l'onde.

In [ ]:
magnitude = None

In [ ]:
frequency = np.linspace(0, sr, len(magnitude))

In [ ]:
plt.plot(frequency, magnitude)
plt.xlabel("Frequency")
plt.ylabel("Magnitude")
plt.show()

In [ ]:
left_frequency = frequency[:int(len(frequency)/2)]
left_magnitude = magnitude[:int(len(frequency)/2)]

In [ ]:
plt.plot(left_frequency, left_magnitude)
plt.xlabel("Frequency")
plt.ylabel("Magnitude")
plt.show()

In [ ]:
n_fft=2048 # Number of time for each sample
hop_length = 512 # the amount we slide to the right

Utilisez la fonction `stft` pour appliquer la Short-time Fourier transform sur le `signal`, utilisez la hyperparamètre `n_fft` et `hop_length` proposé.

In [ ]:
stft = None

Appliquez la fonction `abs` de `numpy` pour obtenir le spectogram.

In [ ]:
spectogram = None

Utilisez la fonction `specshow` de `librosa` pour afficher le spectogram.

In [ ]:
None
plt.xlabel("Time")
plt.ylabel("Frequency")
plt.colorbar()
plt.show()

Ce n'est pas très facile à interpréter.

Utilisez la fonction `amplitude_to_db` pour appliquer le logatirhme à `spectogram` et obtenir le résultat en db.

In [ ]:
log_spectogram = None

Utilisez à nouveau la fonction `specshow` pour afficher le résultat obtenu.

In [ ]:
None
plt.xlabel("Time")
plt.ylabel("Frequency")
plt.colorbar()
plt.show()

Utilisez la fonction `mfcc` de `librosa` pour appliquer le Mel-frequency cepstral coefficients de l'onde.

Prenez un `n_mfcc` de 13.

In [ ]:
MFFCs = None

Utilisez la fonction `specshow` pour visualiser le résultat de la fonction `mfcc`.

In [ ]:
None
plt.xlabel("Time")
plt.ylabel("MFCC")
plt.colorbar()
plt.show()

# Création du jeu de données

Regardez le nombre total de genre à classifier.

In [ ]:
for i, (dirpath, dirnames, filesnames) in enumerate(os.walk("drive/MyDrive/Music_genre_classification/genres_original/")):

  # ensure that we're not at the root level
  if dirpath != "drive/MyDrive/Music_genre_classification/genres_original/":
    dirpath_components = dirpath.split("/")
    semantic_label = dirpath_components[-1]
    print(semantic_label)

In [ ]:
# sample rate
SAMPLE_RATE = 22050

# Longueur de chaque morceau du jeu de données
DURATION = 30

# durée de chaque segment de chanson
SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION

La fonction suivante va diviser chaque son en segment 5 segments et calculer le mfcc sur chacun de ces segments.

Ces résultats seront stocker dans un dictionnaire qui sera sauvegarder en json.

Cette opération peut prendre du temps, donc je vous laisse à disposition le résultat final de cette opération.

In [ ]:
def save_mfcc(dataset_path, json_path, n_mfcc=13, n_fft=2048, hop_length=512, num_segments=5):
  """

  """
  # dictionary to store data
  data = {
      "mapping": [],
      "labels": [],
      "mfcc": []
  }

  num_samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
  expected_num_mfcc_vectors_per_segment = np.ceil(num_samples_per_segment / hop_length)

  # Loop through all the genres

  for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):

    # ensure that we're not at the root level
    if dirpath is not dataset_path:

      # save the semantic label
      dirpath_components = dirpath.split("/")
      semantic_label = dirpath_components[-1]
      data["mapping"].append(semantic_label)
      print('\nProcessing {}'.format(semantic_label))

      # process files for a specific genre
      for f in filenames:
        # load the audio file
        file_path = os.path.join(dirpath, f)
        signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)

        # process segments extracting mfcc and sotring data
        for s in range(num_segments):
          start_sample = num_samples_per_segment * s
          finish_sample = start_sample + num_samples_per_segment

          mfcc = librosa.feature.mfcc(y=signal[start_sample:finish_sample],
                                      sr=sr,
                                      n_fft=n_fft,
                                      n_mfcc=n_mfcc,
                                      hop_length=hop_length)

          mfcc = mfcc.T

          # store mfcc for segment if it has the expected length
          if len(mfcc) == expected_num_mfcc_vectors_per_segment:
            data["mfcc"].append(mfcc.tolist())
            data["labels"].append(i-1)
            print("{}, segment:{}".format(file_path, s))

  with open(json_path, "w") as fp:
    json.dump(data, fp, indent=4)


Calcul, création et sauvegarde du jeu de données.

In [ ]:
#data = save_mfcc(dataset_path="drive/MyDrive/Music_genre_classification/genres_original/",
#          json_path="drive/MyDrive/Music_genre_classification/genres_original/data.json",
#          num_segments=10)

Chargement des données

In [ ]:
def load_data(dataset_path):
  with open(dataset_path, "r") as fp:
    data = json.load(fp)

    # Convert lists into numpy arrays
    inputs = np.array(data["mfcc"])
    targets = np.array(data["labels"])

    return inputs, targets

In [ ]:
inputs, targets = load_data("drive/MyDrive/Music_genre_classification/genres_original/data.json")

In [ ]:
inputs.shape

In [ ]:
targets.shape

Utilisez la fonction `train_test_split` pour séparer le jeu de données en jeu de d'entraînement et de test à partir de `inputs` et `targets`.

In [ ]:
inputs_train, inputs_test, targets_train, targets_test = None

# Deep learning classique

# Créer le générateur

Création d'un générateur adapté à notre application.

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, x_train, y_train):
        self.input = x_train
        self.output = y_train

    def __len__(self):
        return len(self.output)

    def __getitem__(self, idx):
        batch_input = self.input[idx, :, :]
        batch_output = self.output[idx]

        return batch_input, batch_output

In [ ]:
x_training = CustomDataset(torch.from_numpy(np.float32(inputs_train)),
                                 torch.from_numpy(targets_train))

Utilisez la fonction `DataLoader` pour créer le générateur avec une taille de batch de 32.

In [ ]:
dataloader_train = None

Vérification que le générateur fonction bien.

In [ ]:
for x, y in dataloader_train:
  print(x.shape)
  print(y.shape)
  break

In [ ]:
x_testing = CustomDataset(torch.from_numpy(np.float32(inputs_test)),
                                 torch.from_numpy(targets_test))

Utilisez la fonction `DataLoader` pour créer le générateur de test.

In [ ]:
dataloader_test = None

Vérification du générateur de test

In [ ]:
for x, y in dataloader_test:
  print(x.shape)
  print(y.shape)
  break

# Entraînement du modèle

## Fonction d'entraînement

In [ ]:
def number_of_good_prediction(prediction:float, target:int):
  one_hot_prediction = np.argmax(prediction, axis=1)
  return np.sum(one_hot_prediction == target)

In [ ]:
def step(model:torch.nn.Sequential,
         opt:torch.optim,
         criterion:torch.nn.modules.loss,
         x_train:torch.Tensor,
         y_train:torch.Tensor,
         metric_function)->tuple:
  """
  Executes a single training step for a PyTorch model.
  This function performs a forward pass to compute the model's predictions, calculates
  the loss between predictions and actual target values, computes gradients for each
  model parameter, and updates the parameters using the optimizer.

  Args:
      model (torch.nn.Sequential): The PyTorch model to train.
      optimizer (torch.optim.Optimizer): Optimizer used to update the model's parameters.
      criterion (torch.nn.modules.loss._Loss): Loss function used to compute the error.
      x_train (torch.Tensor): Input training data (features).
      y_train (torch.Tensor): Ground truth labels or target values for the training data.
  Returns:
      tuple: The updated model and the computed loss for the current step.
  """

  # Réinitialisez les gradients d'optimizer à zéro avec la méthode 'zero_grad'
  opt.zero_grad()

  # Calculez les prédiction sur le jeu d'entraînement avec la méthode 'froward'
  prediction = model.forward(x_train)

  # Calculez l'erreur de prédiction avec 'criterion'
  loss = criterion(prediction, y_train)

  performance = metric_function(prediction.detach().numpy(), y_train.detach().numpy())

  # Calculez les gradients avec la méthode 'backward'
  loss.backward()

  # Mettre à jour les paramètres du modèle avec la méthode 'step'
  opt.step()

  return model, loss, performance

In [ ]:
def fit(model, optimizer, criterion, epoch, trainloader, testloader, metric_function):
    epoch = epoch
    history_train_loss = []
    history_test_loss = []
    history_train_metrics = []
    history_test_metrics = []

    for e in range(epoch) :

      train_loss_batch = 0
      test_loss_batch = 0
      train_metric_batch = 0
      test_metric_batch = 0

      for images, labels in trainloader:

        # mise à jour des poids avec la fonction 'step'
        model, train_loss, train_performance = step(model, optimizer, criterion, images, labels, metric_function)

        train_loss_batch += train_loss.detach().numpy()

        train_metric_batch += train_performance

      for images, labels in testloader:

        prediction = model.forward(images)

        test_loss = criterion(prediction, labels)

        test_metric_batch += metric_function(prediction.detach().numpy(), labels.detach().numpy())

        test_loss_batch += test_loss.detach().numpy()

      train_loss_batch /= len(trainloader.sampler)
      test_loss_batch /= len(testloader.sampler)

      train_metric_batch /= len(trainloader.sampler)
      test_metric_batch /= len(testloader.sampler)

      # Sauvegarde des coûts d'entraînement avec append
      history_train_loss = np.append(history_train_loss, train_loss_batch)
      history_test_loss = np.append(history_test_loss, test_loss_batch)

      # Sauvegarde des coûts d'entraînement avec append
      history_train_metrics = np.append(history_train_metrics, train_metric_batch)
      history_test_metrics = np.append(history_test_metrics, test_metric_batch)

      print(f'epoch : {e}/{epoch}')
      print('train_loss : '+str(np.squeeze(train_loss_batch))+ ' test_loss : '+str(np.squeeze(test_loss_batch)))
      print('train_metric : '+str(np.squeeze(train_metric_batch))+ ' test_metric : '+str(np.squeeze(test_metric_batch)))
      print('-------------------------------------------------------------------------------------------------')

    return model, history_train_loss, history_test_loss, history_train_metrics, history_test_metrics


## Création de l'architecture

Utilisez la fonction `Sequantial` pour intialiser l'architecture suivante:
- Apppliquer une couche de flatten pour transformer la matrice d'entrée en vecteur.
- Créez une couche dense de 512 neurones avec la fonction `Linear`,
- Appliquer Relu en fonction d'acitvation,
- Créer une couche dense avec 256 neurones,
- Appliquer le Relu,
- Créer une couche de 10 neurones,
- Appliquer la fonction LogSoftmax.


In [ ]:
model_deep = None

Utilisez la fonction `summary` pour visualiser le modèle.

Attention si vous êtes dans une session avec gpu utilisez en paramètre `device='cpu'`

In [ ]:
None

Utilisez la fonction `NLLLoss` pour fonction de coût.

Utilisez la fonction `Adam` comme optimizer avec un learning rate de 0.001.

In [ ]:
criterion = None
optimizer = None

In [ ]:
epoch = 10

Utilisez la fonction `fit` pour entraîner le modèle.

In [ ]:
model, history_train_loss_deep, history_test_loss_deep, history_train_metrics_deep, history_test_metrics_deep = None

# CNN

On va devoir changer l'opérateur car on doit ajouter un channel pour le CNN.

In [ ]:
x_training = CustomDataset(torch.from_numpy(np.float32(np.expand_dims(inputs_train, axis=1))),
                                 torch.from_numpy(targets_train))

UTilisez la fonction `DataLoader` pour créer le générateur d'entraînement.

In [ ]:
dataloader_train_cnn = None

Vérification du générateur d'entraînement.

In [ ]:
for x, y in dataloader_train_cnn:
  print(x.shape)
  print(y.shape)
  break

In [ ]:
x_testing = CustomDataset(torch.from_numpy(np.float32(np.expand_dims(inputs_test, axis=1))),
                                 torch.from_numpy(targets_test))

Utilisez la fonction `Data Loader` pour initialiser le générateur de test.

In [ ]:
dataloader_test_cnn = None

Vérification du générateur de test.

In [ ]:
for x, y in dataloader_test_cnn:
  print(x.shape)
  print(y.shape)
  break

## Initialisation de l'architecture


Utilisez la fonction `Sequantial` pour initialiser l'architecture suivante:
- Une couche de convolution avec 32 filtres,
- Une couche d'activation Relu,
- Une couche de maxpooling,
- Une couche de convolution avec 64 filtres,
- Une couche d'activation Relu,
- Une couche de maxpooling,
- Une couche de Flatten,
- Une couche Linéaire avec 64 neurones,
- Une couche de Relu,
- Une couche linéaire avec 10 neurones,
- Une couche d'activation Logsoftmax

In [ ]:
model_cnn = None

Utilisez la fonction `summary`pour visualiser le modèle.

In [ ]:
None

Utilisez la fonction `NLLLoss` pour fonction de coût.

Utilisez la fonction `Adam` comme optimizer avec un learning rate de 0.001.

In [ ]:
criterion = None
optimizer = None

Utilisez la fonction `fit` pour entraîner le modèle.

In [ ]:
model, history_train_loss_cnn, history_test_loss_cnn, history_train_metrics_cnn, history_test_metrics_cnn = None

# RNN

Définir le modèle de la manière suivante.

Dans `__init__` vous allez créer des attributs à la classe reprenant les opérations que vous allez vouloir utiliser dans le modèle.

Dans `foward` vous allez appliquer le graph de calcul que vous voulez utiliser.

L'architecture est la suivante:
- Une couche de RNN,
- Une couche de linéaire qui prend en entrée la dernière couche caché de la séquence et avec 64 neurones,
- Une couche d'activation Relu,
- Une couche linéiare avec 10 neurones,
- Une couche d'activation logsoftmax,

In [ ]:
# Définir le modèle RNN
class RNNModel(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # Couche RNN
        self.rnn = None

        # Couche après le RNN
        self.fc1 = None

        # Couche entièrement connectée pour la sortie
        self.fc2 = None

        self.softmax = None

    def forward(self, x):
        # Initialiser l'état caché avec des zéros
        h0 = None

        # Passer les données dans la couche RNN
        out, h_n = None

        fc1 = None

        fc2 = None

        proba = None
        return proba

In [ ]:
# Définir les dimensions d'entrée et de sortie
input_size = 13
hidden_size = 128
num_layers = 1
output_size = 10

# Initialiser le modèle
model_rnn = RNNModel(input_size, hidden_size, num_layers, output_size)

Utilisez la fonction `summary` pour visualiser le modèle.

In [ ]:
None

Utilisez la fonction `NLLLoss` pour fonction de coût.

Utilisez la fonction `Adam` comme optimizer avec un learning rate de 0.001.

In [ ]:
criterion = None
optimizer = None

Utilisez la fonction `fit` pour entraîner le modèle.

In [ ]:
model, history_train_loss_rnn, history_test_loss_rnn, history_train_metrics_rnn, history_test_metrics_rnn = None

Visualisez les performances des trois architectures.

In [ ]:
#plt.plot(np.arange(epoch), history_train_loss_deep, label='train loss deep')
plt.plot(np.arange(epoch), history_test_loss_deep, label='test loss deep')
#plt.plot(np.arange(epoch), history_train_loss_cnn, label='train loss cnn')
plt.plot(np.arange(epoch), history_test_loss_cnn, label='test loss cnn')
#plt.plot(np.arange(epoch), history_train_loss_rnn, label='train loss rnn')
plt.plot(np.arange(epoch), history_test_loss_rnn, label='test loss rnn')
plt.xlabel('Epochs')
plt.ylabel('loss')
plt.legend(loc='upper left')
plt.legend()
plt.show()

In [ ]:
#plt.plot(history_train_metrics_deep, label='train accuracy deep')
plt.plot(history_test_metrics_deep, label='test accuracy deep')
#plt.plot(history_train_metrics_cnn, label='train accuracy cnn')
plt.plot(history_test_metrics_cnn, label='test accuracy cnn')
#plt.plot(history_train_metrics_rnn, label='train accuracy rnn')
plt.plot(history_test_metrics_rnn, label='test accuracy rnn')
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc='upper left')
plt.show()